In [2]:
!ls data/

drugs               drugs_html_pages.7z


In [3]:
%cd data/drugs

/Users/ariellubonja/prog/drugs-embedding/data/drugs


## Get "Related/similar drugs" URLs from each page

In [4]:
!grep -F 'Related/similar drugs' https:\$\$www.drugs.com\$vizimpro.html

<h2>Related/similar drugs</h2>


In [5]:
from bs4 import BeautifulSoup

In [6]:
html_text = ''
with open("https:$$www.drugs.com$vizimpro.html", 'r') as first_file:
    html_text = first_file.read()

In [7]:
parsed_text = BeautifulSoup(html_text)
# html_text

In [8]:
related_drugs = parsed_text.find("h2", string='Related/similar drugs').parent()
related_drugs

[<h2>Related/similar drugs</h2>,
 <a href="/exkivity.html">Exkivity</a>,
 <a href="/methotrexate.html">methotrexate</a>,
 <a href="/mtm/paclitaxel.html">paclitaxel</a>,
 <a href="/keytruda.html">Keytruda</a>,
 <a href="/mtm/cisplatin.html">cisplatin</a>,
 <a href="/avastin.html">Avastin</a>,
 <a href="/tagrisso.html">Tagrisso</a>]

In [9]:
related_drugs[1]

<a href="/exkivity.html">Exkivity</a>

In [10]:
related_drugs[1]['href']

'/exkivity.html'

In [11]:
BASE_URL = "https://www.drugs.com"

In [ ]:
# Create edgelist in [[source1, destination1], [source2, destination2], ...]
edgelist = []
for i in range(1, len(related_drugs)): # Need to skip the first line with <h2></h2>
    # print(ahref)
    edgelist.append(["https:$$www.drugs.com$vizimpro.html", (BASE_URL+related_drugs[i]['href']).replace("/","$")])

edgelist

[['https:$$www.drugs.com$vizimpro.html',
  'https:$$www.drugs.com$exkivity.html'],
 ['https:$$www.drugs.com$vizimpro.html',
  'https:$$www.drugs.com$methotrexate.html'],
 ['https:$$www.drugs.com$vizimpro.html',
  'https:$$www.drugs.com$mtm$paclitaxel.html'],
 ['https:$$www.drugs.com$vizimpro.html',
  'https:$$www.drugs.com$keytruda.html'],
 ['https:$$www.drugs.com$vizimpro.html',
  'https:$$www.drugs.com$mtm$cisplatin.html'],
 ['https:$$www.drugs.com$vizimpro.html', 'https:$$www.drugs.com$avastin.html'],
 ['https:$$www.drugs.com$vizimpro.html',
  'https:$$www.drugs.com$tagrisso.html']]

### Repeat the above for all pages

In [ ]:
# If it exists, return the related drug links in the current page
# html_text_string: simple string resulting from filename.read()
def find_related_drug_links(html_text_string, show_errors=True):
    parsed_text = BeautifulSoup(html_text_string)
    try:
        related_drugs_parent = parsed_text.find("h2", string='Related/similar drugs').parent()
        BASE_URL = "https://www.drugs.com"

        related_urls = []
        for i in range(1, len(related_drugs_parent)): # Need to skip the first line with <h2></h2>
            # print(ahref)
            related_urls.append(BASE_URL+related_drugs_parent[i]['href'])

        return related_urls
    except AttributeError:
        if show_errors:
            print("Page has no Related drugs section, skipping: ", parsed_text.find("title"))
        return []

## Create graph - using url (replaced / with $ due to path issues) as node, and Related drugs as edges

In [ ]:
import os

# Create edgelist in [[source1, destination1], [source2, destination2], ...]
# Reads current directory, where each drug's html page is saved
def create_edgelist_graph():
    print("Creating Graph edgelist from Drugs files at ", os.getcwd())

    edgelist = []
    j=0
    NR_FILES = len(os.listdir())
    for filename in os.listdir():
        if j % 100 == 0:
            print("Processed ", j, " / ", NR_FILES)

        # Cannot use / in linux filepaths - it means folder
        filename_replaced = filename.replace("/","$")

        # Load drug.html file, parse html
        html_text = ''
        with open(filename_replaced, 'r') as first_file:
            html_text = first_file.read()

        # Find the Related Drugs section in the page, if it exists
        related_drugs = find_related_drug_links(html_text, show_errors=False)

        for i in range(len(related_drugs)): # Need to skip the first line with <h2></h2>
            # print(related_drugs)
            edgelist.append([filename_replaced, related_drugs[i].replace("/","$")])

        j += 1

    return edgelist

In [ ]:
edgelist = create_edgelist_graph()

Creating Graph edgelist from Drugs files at  /content/drive/MyDrive/drugs-com/drugs
Processed  0  /  15637
Processed  100  /  15637
Processed  200  /  15637
Processed  300  /  15637
Processed  400  /  15637
Processed  500  /  15637
Processed  600  /  15637
Processed  700  /  15637
Processed  800  /  15637
Processed  900  /  15637
Processed  1000  /  15637
Processed  1100  /  15637
Processed  1200  /  15637
Processed  1300  /  15637
Processed  1400  /  15637
Processed  1500  /  15637
Processed  1600  /  15637
Processed  1700  /  15637
Processed  1800  /  15637
Processed  1900  /  15637
Processed  2000  /  15637
Processed  2100  /  15637
Processed  2200  /  15637
Processed  2300  /  15637
Processed  2400  /  15637
Processed  2500  /  15637
Processed  2600  /  15637
Processed  2700  /  15637
Processed  2800  /  15637
Processed  2900  /  15637
Processed  3000  /  15637
Processed  3100  /  15637
Processed  3200  /  15637
Processed  3300  /  15637
Processed  3400  /  15637
Processed  3500  /

In [ ]:
edgelist[:10]

[['https:$$www.drugs.com$cons$benzamycin-pak-topical.html',
  'https:$$www.drugs.com$doxycycline.html'],
 ['https:$$www.drugs.com$cons$benzamycin-pak-topical.html',
  'https:$$www.drugs.com$mtm$clindamycin-topical.html'],
 ['https:$$www.drugs.com$cons$benzamycin-pak-topical.html',
  'https:$$www.drugs.com$mtm$erythromycin-topical.html'],
 ['https:$$www.drugs.com$cons$benzamycin-pak-topical.html',
  'https:$$www.drugs.com$tetracycline.html'],
 ['https:$$www.drugs.com$cons$benzamycin-pak-topical.html',
  'https:$$www.drugs.com$tazorac.html'],
 ['https:$$www.drugs.com$cons$qualisone.html',
  'https:$$www.drugs.com$otezla.html'],
 ['https:$$www.drugs.com$cons$qualisone.html',
  'https:$$www.drugs.com$taltz.html'],
 ['https:$$www.drugs.com$cons$qualisone.html',
  'https:$$www.drugs.com$prednisone.html'],
 ['https:$$www.drugs.com$cons$qualisone.html',
  'https:$$www.drugs.com$doxycycline.html'],
 ['https:$$www.drugs.com$cons$qualisone.html',
  'https:$$www.drugs.com$methotrexate.html']]

In [ ]:
import pandas as pd

edgelist_df = pd.DataFrame(edgelist)

In [ ]:
edgelist_df.head()

,0,1
0,https:$$www.drugs.com$cons$benzamycin-pak-topi...,https:$$www.drugs.com$doxycycline.html
1,https:$$www.drugs.com$cons$benzamycin-pak-topi...,https:$$www.drugs.com$mtm$clindamycin-topical....
2,https:$$www.drugs.com$cons$benzamycin-pak-topi...,https:$$www.drugs.com$mtm$erythromycin-topical...
3,https:$$www.drugs.com$cons$benzamycin-pak-topi...,https:$$www.drugs.com$tetracycline.html
4,https:$$www.drugs.com$cons$benzamycin-pak-topi...,https:$$www.drugs.com$tazorac.html


In [ ]:
# Save to CSV
edgelist_df.to_csv("drugs_html_edgelist.csv", index=False)